## Final Project Submission

### GROUP 4

Please fill out:
* Students name: 
* Student pace: **part time**
* Scheduled project review date/time: 
* Instructor name: **Brian Chacha**
* Blog post URL:


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

#  Movie Studio Strategy: Data-Driven Decisions for Box Office Success

##  Introduction

In an increasingly competitive content landscape, companies are investing heavily in original video production. Our company is launching a new movie studio and aims to enter this space strategically. However, lacking prior experience in film production, we must rely on data to identify what makes a movie successful.

This project leverages real-world movie data to explore what types of films perform best at the box office. Through structured analysis, we aim to generate insights that can inform content creation, budgeting, casting, and release strategies.

---
## ❓ Business Understanding

###  Business Objective

Our primary business question is:

> **What types of films should our company produce to maximize success at the box office?**

To answer this, we explore several supporting questions:

1. **What are the current trends by genre in the box office?**  
   - Which genres dominate the market?
   - Which ones generate the highest gross and ROI?

2. **What budget ranges are the most profitable?**  
   - How can we optimize spending without compromising success?

3. **Which directors and actors consistently attract larger audiences?**  
   - Who are the “bankable names” we should work with?

4. **What is the average length (runtime) of films that perform well?**  
   - Is there an ideal duration for maximizing audience engagement?

5. **When is the best time to release a movie?**  
   - Are there specific months or seasons that lead to higher revenues?

Our goal is to convert findings from these questions into **three actionable business recommendations**.

---